# Binary QSAR in Python - LabMol

Developed by: 
#### José Teófilo Moreira Filho

#  <font color='white'> Model building with featmorgan fingerprint and RF, SVM and LightGBM </font>

In [ ]:
import datetime
now = datetime.datetime.now()
print("date of creation:\n", now)

In [ ]:
# Importing packages 
from rdkit import Chem, DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import PandasTools

import numpy as np
from numpy import sqrt
from numpy import argmax

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from pandas import DataFrame

import os

import xgboost as xgb
import lightgbm as lgb

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef, roc_curve, precision_recall_curve, roc_auc_score, make_scorer
from sklearn.metrics import f1_score, balanced_accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import auc as mauc
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_digits
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import cross_val_predict

from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.metrics import geometric_mean_score
from imblearn.under_sampling import TomekLinks

import multiprocessing

from skopt import BayesSearchCV

from mordred import Calculator, descriptors

## Check initial number of compounds

In [ ]:
# Reading molecules and activity (0 and 1) from SDF
fname = "../data/1224857_bruno_imbalanced_prep.sdf"

In [ ]:
df0 = PandasTools.LoadSDF(fname, smilesName='SMILES', includeFingerprints=False)

In [ ]:
df0.columns

In [ ]:
df0["is_active"] = np.where(df0["Outcome"] == "Active", 1,0)

In [ ]:
df0.head()

In [ ]:
target_count = df0.is_active.value_counts()
print('Class 0:', target_count[0])
print('Class 1:', target_count[1])
print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')

target_count.plot(kind='bar', title='Count (target)');

In [ ]:
#Save the table as SD file
#PandasTools.WriteSDF(df0, '../data/w2_isactive_5445.sdf', properties=df0.columns)

## Fingerprint and descriptors calculation

In [ ]:
# Reading molecules and activity (0 and 1) from SDF
fname = "../data/1224857_bruno_imbalanced_prep.sdf"

mols = []
y = []
for mol in Chem.SDMolSupplier(fname):
    if mol is not None:
        mols.append(mol)
        y.append(mol.GetIntProp("is_active")) # target column

In [ ]:
type(y)

In [ ]:
y = pd.DataFrame(y)

In [ ]:
y = y.iloc[:,-1:].values.ravel()

In [ ]:
# Calculate descriptors (fingerprints) and convert them into numpy array

# generate binary FeatMorgan fingerprint with radius 2
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2, 2048, useFeatures=True) for m in mols]

def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)
# Convert fingerprint to numpy array
x_fp = rdkit_numpy_convert(fp)

In [ ]:
# Convert fingerprint to Pandas DataFram,e
x_fp = pd.DataFrame(x_fp)

In [ ]:
len(x_fp)

In [ ]:
# create descriptor calculator with all descriptors
calc = Calculator(descriptors, ignore_3D=True)

In [ ]:
# Check the number of descriptors
len(calc.descriptors)

In [ ]:
# mordred descriptor calculation
df = calc.pandas(mols)

In [ ]:
# Print fisrt five rows
df.head()

In [ ]:
# Check the number of compounds and descriptors
df.shape

In [ ]:
# Check the columns
df.columns

### Scale data

In [ ]:
# scale data between 0 and 1
from sklearn import preprocessing

x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df2 = pd.DataFrame(x_scaled)

In [ ]:
# Check the number of compounds and descriptors
df2.shape

In [ ]:
# Print fisrt five rows of the scaled DataFrame of descriptors
df2.head()

In [ ]:
# Print fisrt five rows of the first DataFrame of descriptors
df.head()

In [ ]:
# Recuperate the name of columns lost during scaling
df2.columns=df.columns.values

In [ ]:
# Print fisrt five rows 
df2.head()

### Remove columns with NaN

In [ ]:
# Check the existence of NaN values
df2.isnull().values.any()

In [ ]:
# Drop the columns with NaN values
df3 = df2.dropna(axis = 1, how ='any')

In [ ]:
# Check the number of compounds and descriptors
df3.shape

In [ ]:
# Check the existence of NaN values
df3.isnull().values.any()

In [ ]:
# Print fisrt five rows 
df3.head()

### Remove High correlated features

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
# Constant descriptors
constant_filter = VarianceThreshold(threshold=0.01)  
constant_filter.fit(df3) 

In [ ]:
# Number of not constant descriptors
len(df3.columns[constant_filter.get_support()])

In [ ]:
# Number of constant descriptors
constant_columns = [column for column in df3.columns
                    if column not in df3.columns[constant_filter.get_support()]]

print(len(constant_columns))

In [ ]:
# Remove constant columns
df4 = df3.drop(labels=constant_columns, axis=1)

In [ ]:
# Print fisrt five rows 
df4.head()

In [ ]:
# Check the number of compounds and descriptors
df4.shape

In [ ]:
# Calculate correlation
correlated_features = set()
correlation_matrix = df4.corr()

In [ ]:
threshold = 0.90

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

In [ ]:
# Remove correlated columns
df_clean = df4.drop(labels=correlated_features, axis=1)

In [ ]:
# Print fisrt five rows 
df_clean.head()

### Join fingerprint to Mordred descriptors

In [ ]:
# Join fingerprint to Mordred descriptors
x = df_clean.join(x_fp)

In [ ]:
# Print fisrt five rows
x.head()

In [ ]:
# Check the number of compounds and descriptors
x.shape

In [ ]:
x = np.array(x)

In [ ]:
print(len(x))
type(x)

In [ ]:
print(len(y))
type(y)

## Data splitting

In [ ]:
#Create folds for cross-validation
cv = StratifiedKFold(n_splits=5, random_state=42)

## RF Model building - Bayesian hyperparameter search  

In [ ]:
scorer = make_scorer(geometric_mean_score)

# log-uniform: understand as search over p = exp(x) by varying x
opt_rf = BayesSearchCV(
    RandomForestClassifier(),
    {'max_features': ['auto', 'sqrt'],
    'n_estimators': [100, 1000],
    "max_depth": [2, 100],
    'min_samples_leaf': [1,20], 
    'min_samples_split': [2, 20]
    },
    n_iter=50, # Number of parameter settings that are sampled
    cv=cv,
    scoring = scorer,
    verbose=0,
    refit= True, # Refit the best estimator with the entire dataset.
    random_state=42, 
    n_jobs = -1
)

opt_rf.fit(x, y)

print("Best parameters: %s" % opt_rf.best_params_)

## 5-fold cross-validation

In [ ]:
probs_classes = []
#indexes = []
y_test_all = []

for train_index, test_index in cv.split(x, y):
    rf_clf = RandomForestClassifier(**opt_rf.best_params_) # model with best parameters
    X_train_folds = x[train_index] # descritors train split
    y_train_folds = np.array(y)[train_index.astype(int)] # label train split
    X_test_fold = x[test_index] # descritors test split
    y_test_fold = np.array(y)[test_index.astype(int)] # label test split
    
    
    rf_clf.fit(X_train_folds, y_train_folds) # train fold
    y_pred = rf_clf.predict_proba(X_test_fold) # test fold
    probs_classes.append(y_pred) # all predictions for test folds
    y_test_all.append(y_test_fold) # all folds' labels 
#   indexes.append(test_index) # all tests indexes

## Check performance of each fold

In [ ]:
# Get predictions of each fold
fold_1_pred = (probs_classes[0][:, 1] > 0.5).astype(int)
fold_2_pred = (probs_classes[1][:, 1] > 0.5).astype(int)
fold_3_pred = (probs_classes[2][:, 1] > 0.5).astype(int)
fold_4_pred = (probs_classes[3][:, 1] > 0.5).astype(int)
fold_5_pred = (probs_classes[4][:, 1] > 0.5).astype(int)

In [ ]:
# Get experimental values of each fold
fold_1_exp = y_test_all[0]
fold_2_exp = y_test_all[1]
fold_3_exp = y_test_all[2]
fold_4_exp = y_test_all[3]
fold_5_exp = y_test_all[4]

In [ ]:
bacc1 = metrics.balanced_accuracy_score(fold_1_exp, fold_1_pred) # balanced accuracy fold 1
bacc2 = metrics.balanced_accuracy_score(fold_2_exp, fold_2_pred) # balanced accuracy fold 2
bacc3 = metrics.balanced_accuracy_score(fold_3_exp, fold_3_pred) # balanced accuracy fold 3
bacc4 = metrics.balanced_accuracy_score(fold_4_exp, fold_4_pred) # balanced accuracy fold 4
bacc5 = metrics.balanced_accuracy_score(fold_5_exp, fold_5_pred) # balanced accuracy fold 5
print("Balanced accuracy (fold 1) = ", bacc1)
print("Balanced accuracy (fold 2) = ", bacc2)
print("Balanced accuracy (fold 3) = ", bacc3)
print("Balanced accuracy (fold 4) = ", bacc4)
print("Balanced accuracy (fold 5) = ", bacc5)

## Check mean performance of folds

In [ ]:
probs_classes = np.concatenate(probs_classes)    
y_experimental = np.concatenate(y_test_all)

In [ ]:
# Uncalibrated model predictions
pred_rf = (probs_classes[:, 1] > 0.5).astype(int)

### Statistics - featmorgan-RF

In [ ]:
def calc_statistics(y,pred):
    # save confusion matrix and slice into four pieces
    confusion = confusion_matrix(y, pred)
    #[row, column]
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    
    # Plot confusion
    #plt.figure(figsize=(5,5))
    #sns.heatmap(confusion, annot=True, fmt=".0f", linewidths=.5, square = True, cmap = 'Blues_r');
    #plt.ylabel('Actual label');
    #plt.xlabel('Predicted label');
    #title = "Confusion matrix"
    #plt.title(title, size = 15);
    
    # calc statistics
    classification_error = 1 - accuracy_score(y, pred) #Classification error or misclassification rate
    accuracy = accuracy_score(y, pred) #accuracy
    mcc = matthews_corrcoef(y, pred) #mcc
    kappa = cohen_kappa_score(y, pred) #kappa
    sensitivity = recall_score(y, pred) #Sensitivity
    specificity = TN / (TN + FP) #Specificity
    false_positive_rate = FP / float(TN + FP) #False positive rate (alfa)
    false_negative_rate = FN / float(TP+FN) #False negative rate (beta)
    precision = TP / float(TP + FP) #Precision
    positive_pred_value = TP / float(TP + FP) #PPV
    negative_pred_value = TN / float(TN + FN) #NPV
    auc = roc_auc_score(y, pred) #AUC
    bacc = balanced_accuracy_score(y, pred) # balanced accuracy
    f1 = f1_score(y, pred) # F1-score

    print("Accuracy = ", accuracy)
    print("MCC = ", mcc)
    print("Kappa = ", kappa)
    print("Sensitivity = ", sensitivity)
    print("Specificity = ", specificity)
    print("Precision = ", precision)
    print("PPV = ", positive_pred_value)
    print("NPV = ", negative_pred_value)
    print("False positive rate = ", false_positive_rate)
    print("False negative rate = ", false_negative_rate)
    print("AUC = ",roc_auc_score(y, pred))
    print("Classification error = ", classification_error)
    print("Balanced accuracy = ", bacc)
    print("F1-score = ", f1)
    
    #converting calculated metrics into a pandas dataframe to compare all models at the final
    statistics = pd.DataFrame({'Bal-acc': bacc, "Sensitivity": sensitivity, "Specificity": specificity,"PPV": positive_pred_value, 
           "NPV": negative_pred_value, 'Kappa': kappa, 'AUC': auc, 'MCC': mcc, 'Accuracy': accuracy, 
           "Classification error": classification_error,"False positive rate": false_positive_rate, 
           "False negative rate": false_negative_rate, "Precision": precision, 'F1-score': f1,}, index=[0])
    return(statistics)

In [ ]:
statistics = calc_statistics(y_experimental, pred_rf)

In [ ]:
#converting calculated metrics into a pandas dataframe to save a xls
model_type = "hybrid_mordred_featmorgan_r2_2048_rf"

result_type = "uncalibrated"

metrics_rf_uncalibrated = statistics
metrics_rf_uncalibrated['model'] = model_type
metrics_rf_uncalibrated['result_type'] = result_type
metrics_rf_uncalibrated

### Check model calibatrion

In [ ]:
# keep probabilities for the positive outcome only
probs = probs_classes[:, 1]
# reliability diagram
fop, mpv = calibration_curve(y_experimental, probs, n_bins=10)
# plot perfectly calibrated
plt.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
plt.plot(mpv, fop, marker='.')
#plt.show()
plt.savefig('../results/calibration_model_hybrid_mordred_featmorgan_r2_2048_rf_1224857_bruno_imbalanced.png', dpi=300)

### Use ROC-Curve and Gmean to select a threshold for calibration

In [ ]:
# keep probabilities for the positive outcome only
yhat = probs_classes[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(y_experimental, yhat)
# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='RF')
plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
#plt.show()
plt.savefig('../results/calibration_curve_hybrid_mordred_featmorgan_r2_2048_rf_1224857_bruno_imbalanced.png', dpi=300)

In [ ]:
# Record the threshold in a variable
threshold_roc = thresholds[ix]
print(threshold_roc)

### Optimal Threshold for Precision-Recall Curve

In [ ]:
# keep probabilities for the positive outcome only
yhat = probs_classes[:, 1]
# calculate precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_experimental, yhat)
# convert to f score
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

In [ ]:
# Record the threshold in a variable
threshold_prc = thresholds[ix]
print(threshold_prc)

### Statistics model calibrated - Choose the best calibration method before

In [ ]:
# Select the best threshold to distinguishthe classes
pred_rf = (probs_classes[:, 1] > threshold_roc).astype(int)
# pred_rf = (probs_classes[:, 1] > threshold_prc).astype(int)

In [ ]:
statistics = calc_statistics(y_experimental, pred_rf)

In [ ]:
#converting calculated metrics into a pandas dataframe to save a xls
model_type = "hybrid_mordred_featmorgan_r2_2048_rf"

result_type = "calibrated"

metrics_rf_calibrated = statistics
metrics_rf_calibrated['model'] = model_type
metrics_rf_calibrated['result_type'] = result_type
metrics_rf_calibrated['calibration_threshold'] = threshold_roc
metrics_rf_calibrated

In [ ]:
# Saving the dataframe as excel file
#metrics_rf_calibrated.to_excel("../results/model_binary_metrics_rf_featmorgan_r2_gmean_mpro_newdata_thres028.xlsx", sheet_name= "Sheet1")

### Save model

In [ ]:
#Save model - pkl file
joblib.dump(opt_rf, "../models/model_binary_hybrid_mordred_featmorgan_r2_2048_rf_gmean_1224857_bruno_imbalanced.pkl", compress=3)

## SVM Model building - Bayesian hyperparameter search  

In [ ]:
scorer = make_scorer(geometric_mean_score)

# log-uniform: understand as search over p = exp(x) by varying x
opt_svm = BayesSearchCV(
    SVC(probability=True),
    {
        'C': (1e-6, 1e+6, 'log-uniform'),
        'gamma': (1e-6, 1e+1, 'log-uniform'),
        'kernel': ['rbf'],  # categorical parameter | ['linear', 'poly', 'rbf'] to test all kernels
    },
    n_iter=50, # Number of parameter settings that are sampled
    cv=cv,
    scoring = scorer,
    refit = True, # Refit the best estimator with the entire dataset.
    random_state=42,
    n_jobs = -1
)

opt_svm.fit(x, y)

print("Best parameters: %s" % opt_svm.best_params_)

## 5-fold cross-validation

In [ ]:
probs_classes = []
#indexes = []
y_test_all = []

for train_index, test_index in cv.split(x, y):
    svm_clf = SVC(**opt_svm.best_params_, probability=True) # model with best parameters
    X_train_folds = x[train_index] # descritors train split
    y_train_folds = np.array(y)[train_index.astype(int)] # label train split
    X_test_fold = x[test_index] # descritors test split
    y_test_fold = np.array(y)[test_index.astype(int)] # label test split
    
    
    svm_clf.fit(X_train_folds, y_train_folds) # train fold
    y_pred = svm_clf.predict_proba(X_test_fold) # test fold
    probs_classes.append(y_pred) # all predictions for test folds
    y_test_all.append(y_test_fold) # all folds' labels 
#  indexes.append(test_index) # all tests indexes

## Check performance of each fold

In [ ]:
# Get predictions of each fold
fold_1_pred = (probs_classes[0][:, 1] > 0.5).astype(int)
fold_2_pred = (probs_classes[1][:, 1] > 0.5).astype(int)
fold_3_pred = (probs_classes[2][:, 1] > 0.5).astype(int)
fold_4_pred = (probs_classes[3][:, 1] > 0.5).astype(int)
fold_5_pred = (probs_classes[4][:, 1] > 0.5).astype(int)

In [ ]:
# Get experimental values of each fold
fold_1_exp = y_test_all[0]
fold_2_exp = y_test_all[1]
fold_3_exp = y_test_all[2]
fold_4_exp = y_test_all[3]
fold_5_exp = y_test_all[4]

In [ ]:
bacc1 = metrics.balanced_accuracy_score(fold_1_exp, fold_1_pred) # balanced accuracy fold 1
bacc2 = metrics.balanced_accuracy_score(fold_2_exp, fold_2_pred) # balanced accuracy fold 2
bacc3 = metrics.balanced_accuracy_score(fold_3_exp, fold_3_pred) # balanced accuracy fold 3
bacc4 = metrics.balanced_accuracy_score(fold_4_exp, fold_4_pred) # balanced accuracy fold 4
bacc5 = metrics.balanced_accuracy_score(fold_5_exp, fold_5_pred) # balanced accuracy fold 5
print("Balanced accuracy (fold 1) = ", bacc1)
print("Balanced accuracy (fold 2) = ", bacc2)
print("Balanced accuracy (fold 3) = ", bacc3)
print("Balanced accuracy (fold 4) = ", bacc4)
print("Balanced accuracy (fold 5) = ", bacc5)

## Check mean performance of folds

In [ ]:
probs_classes = np.concatenate(probs_classes)    
y_experimental = np.concatenate(y_test_all)

In [ ]:
# Uncalibrated model predictions
pred_svm = (probs_classes[:, 1] > 0.5).astype(int)

### Statistics - featmorgan-SVM

In [ ]:
# Uncalibrated model predictions
pred_svm = (probs_classes[:, 1] > 0.5).astype(int)
len(y_experimental)

In [ ]:
statistics = calc_statistics(y_experimental, pred_svm)

In [ ]:
#converting calculated metrics into a pandas dataframe to save a xls
model_type = "hybrid_mordred_morgan_r2_2048_svm"

result_type = "uncalibrated"

metrics_svm_uncalibrated = statistics
metrics_svm_uncalibrated['model'] = model_type
metrics_svm_uncalibrated['result_type'] = result_type
metrics_svm_uncalibrated

### Check model calibatrion

In [ ]:
# keep probabilities for the positive outcome only
probs = probs_classes[:, 1]
# reliability diagram
fop, mpv = calibration_curve(y_experimental, probs, n_bins=10)
# plot perfectly calibrated
plt.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
plt.plot(mpv, fop, marker='.')
#plt.show()
plt.savefig('../results/calibration_model_hybrid_mordred_featmorgan_r2_2048_svm_1224857_bruno_imbalanced.png', dpi=300)

### Use ROC-Curve and Gmean to select a threshold for calibration

In [ ]:
# keep probabilities for the positive outcome only
yhat = probs_classes[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(y_experimental, yhat)
# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='SVM')
plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
#plt.show()
plt.savefig('../results/calibration_curve_hybrid_mordred_featmorgan_r2_2048_svm_1224857_bruno_imbalanced.png', dpi=300)

In [ ]:
# Record the threshold in a variable
threshold_roc = thresholds[ix]
print(threshold_roc)

### Optimal Threshold for Precision-Recall Curve

In [ ]:
# keep probabilities for the positive outcome only
yhat = probs_classes[:, 1]
# calculate precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_experimental, yhat)
# convert to f score
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

In [ ]:
# Record the threshold in a variable
threshold_prc = thresholds[ix]
print(threshold_prc)

### Statistics model calibrated - Choose the best calibration method before

In [ ]:
# Select the best threshold to distinguishthe classes
pred_svm = (probs_classes[:, 1] > threshold_roc).astype(int)
# pred_svm = (probs_classes[:, 1] > threshold_prc).astype(int)

In [ ]:
statistics = calc_statistics(y_experimental, pred_svm)

In [ ]:
#converting calculated metrics into a pandas dataframe to save a xls
model_type = "hybrid_mordred_featmorgan_r2_2048_svm"

result_type = "calibrated"

metrics_svm_calibrated = statistics
metrics_svm_calibrated['model'] = model_type
metrics_svm_calibrated['result_type'] = result_type
metrics_svm_calibrated['calibration_threshold'] = threshold_roc
metrics_svm_calibrated

In [ ]:
# Saving the dataframe as excel file
#metrics_svm.to_excel("../results/model_binary_metrics_svm_featmorgan_r2_gmean_mpro_newdata_thres039.xlsx", sheet_name= "Sheet1")

### Save model

In [ ]:
#Save model - pkl file
joblib.dump(opt_svm, "../models/model_binary_svm_hybrid_mordred_featmorgan_r2_2048_gmean_1224857_bruno_imbalanced.pkl", compress=3)

## LightGBM Model building - Bayesian hyperparameter search

In [ ]:
scorer = make_scorer(geometric_mean_score)

# log-uniform: understand as search over p = exp(x) by varying x
opt_lgb = BayesSearchCV(lgb.LGBMClassifier(),
                        {'learning_rate': (0.01, 1.0, 'log-uniform'), 
                         'num_leaves': (7, 4095),
                         'n_estimators': (100, 800), 
                         'max_depth': (2, 63),
                         'subsample': (0.4, 1), 
                         'scale_pos_weight': (1, 1000)}, 
                        n_iter = 50, # Number of parameter settings that are sampled
                        cv = cv, 
                        scoring = scorer,
                        refit = True, # Refit the best estimator with the entire dataset.
                        verbose = 0,
                        random_state = 42, 
                        n_jobs = 1)

opt_lgb.fit(x, y)

print("Best parameters: %s" % opt_lgb.best_params_)

## 5-fold cross-validation

In [ ]:
probs_classes = []
#indexes = []
y_test_all = []

for train_index, test_index in cv.split(x, y):
    lgb_clf = lgb.LGBMClassifier(**opt_lgb.best_params_,) # model with best parameters
    X_train_folds = x[train_index] # descritors train split
    y_train_folds = np.array(y)[train_index.astype(int)] # label train split
    X_test_fold = x[test_index] # descritors test split
    y_test_fold = np.array(y)[test_index.astype(int)] # label test split
    
    
    svm_clf.fit(X_train_folds, y_train_folds) # train fold
    y_pred = svm_clf.predict_proba(X_test_fold) # test fold
    probs_classes.append(y_pred) # all predictions for test folds
    y_test_all.append(y_test_fold) # all folds' labels 
#  indexes.append(test_index) # all tests indexes

## Check performance of each fold

In [ ]:
# Get predictions of each fold
fold_1_pred = (probs_classes[0][:, 1] > 0.5).astype(int)
fold_2_pred = (probs_classes[1][:, 1] > 0.5).astype(int)
fold_3_pred = (probs_classes[2][:, 1] > 0.5).astype(int)
fold_4_pred = (probs_classes[3][:, 1] > 0.5).astype(int)
fold_5_pred = (probs_classes[4][:, 1] > 0.5).astype(int)

In [ ]:
# Get experimental values of each fold
fold_1_exp = y_test_all[0]
fold_2_exp = y_test_all[1]
fold_3_exp = y_test_all[2]
fold_4_exp = y_test_all[3]
fold_5_exp = y_test_all[4]

In [ ]:
bacc1 = metrics.balanced_accuracy_score(fold_1_exp, fold_1_pred) # balanced accuracy fold 1
bacc2 = metrics.balanced_accuracy_score(fold_2_exp, fold_2_pred) # balanced accuracy fold 2
bacc3 = metrics.balanced_accuracy_score(fold_3_exp, fold_3_pred) # balanced accuracy fold 3
bacc4 = metrics.balanced_accuracy_score(fold_4_exp, fold_4_pred) # balanced accuracy fold 4
bacc5 = metrics.balanced_accuracy_score(fold_5_exp, fold_5_pred) # balanced accuracy fold 5
print("Balanced accuracy (fold 1) = ", bacc1)
print("Balanced accuracy (fold 2) = ", bacc2)
print("Balanced accuracy (fold 3) = ", bacc3)
print("Balanced accuracy (fold 4) = ", bacc4)
print("Balanced accuracy (fold 5) = ", bacc5)

## Check mean performance of folds

In [ ]:
probs_classes = np.concatenate(probs_classes)    
y_experimental = np.concatenate(y_test_all)

In [ ]:
# Uncalibrated model predictions
pred_lgb = (probs_classes[:, 1] > 0.5).astype(int)

### Statistics - featmorgan-LGB

In [ ]:
pred_lgb = (probs_classes[:, 1] > 0.5).astype(int)

In [ ]:
statistics = calc_statistics(y_experimental, pred_lgb)

In [ ]:
#converting calculated metrics into a pandas dataframe to save a xls
model_type = "hybrid_mordred_featmorgan_r2_2048_lgb"

result_type = "uncalibrated"

metrics_lgb_uncalibrated = statistics
metrics_lgb_uncalibrated['model'] = model_type
metrics_lgb_uncalibrated['result_type'] = result_type
metrics_lgb_uncalibrated

### Check model calibatrion

In [ ]:
# keep probabilities for the positive outcome only
probs = probs_classes[:, 1]
# reliability diagram
fop, mpv = calibration_curve(y_experimental, probs, n_bins=10)
# plot perfectly calibrated
plt.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
plt.plot(mpv, fop, marker='.')
#plt.show()
plt.savefig('../results/calibration_model_hybrid_mordred_featmorgan_r2_2048_lgb_1224857_bruno_imbalanced.png', dpi=300)

### Use ROC-Curve and Gmean to select a threshold for calibration

In [ ]:
# keep probabilities for the positive outcome only
yhat = probs_classes[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(y_experimental, yhat)
# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='LightGBM')
plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
#plt.show()
plt.savefig('../results/calibration_curve_hybrid_mordred_featmorgan_r2_2048_lgb_1224857_bruno_imbalanced.png', dpi=300)

In [ ]:
# Record the threshold in a variable
threshold_roc = thresholds[ix]
print(threshold_roc)

### Optimal Threshold for Precision-Recall Curve

In [ ]:
# keep probabilities for the positive outcome only
yhat = probs_classes[:, 1]
# calculate precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_experimental, yhat)
# convert to f score
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

In [ ]:
# Record the threshold in a variable
threshold_prc = thresholds[ix]
print(threshold_prc)

### Statistics model calibrated - Choose the best calibration method before

In [ ]:
# Select the best threshold to distinguishthe classes
pred_lgb = (probs_classes[:, 1] > threshold_roc).astype(int)
# pred_lgb = (probs_classes[:, 1] > threshold_prc).astype(int)

In [ ]:
statistics = calc_statistics(y_experimental, pred_lgb)

In [ ]:
#converting calculated metrics into a pandas dataframe to save a xls
model_type = "hybrid_mordred_featmorgan_r2_2048_lgb"

result_type = "calibrated"

metrics_lgb_calibrated = statistics
metrics_lgb_calibrated['model'] = model_type
metrics_lgb_calibrated['result_type'] = result_type
metrics_lgb_calibrated['calibration_threshold'] = threshold_roc
metrics_lgb_calibrated

In [ ]:
# Saving the dataframe as excel file
#metrics_lgb.to_excel("../results/model_binary_metrics_lgb_featmorgan_bee_mpro_newdata_gmean_NO_classweight_thres029.xlsx", sheet_name= "Sheet1")

## Compare all models

## Save an excell with all results

In [ ]:
frames = [metrics_rf_uncalibrated, metrics_svm_uncalibrated, metrics_lgb_uncalibrated, 
          metrics_rf_calibrated, metrics_svm_calibrated, metrics_lgb_calibrated]

result = pd.concat(frames)

column_names = ["model", "Bal-acc", "Accuracy", "Sensitivity", "Specificity", "PPV", "NPV", "Kappa", "MCC", "AUC", "F1-score", 
                "Classification error", "False positive rate", "False negative rate", "result_type", "calibration_threshold"]
result = result.reindex(columns=column_names)
result = result.round(2)

result

In [ ]:
# Saving the dataframe as excel file
result.to_excel("../results/metrics_binary_hybrid_mordred_featmorgan_r2_2048_allmodels_1224857_bruno_imbalanced.xlsx", sheet_name= "Sheet1")